# Lets save our data as a huggingface dataset, so it's quick to reuse



In [1]:
# import your package
%load_ext autoreload
%autoreload 2

from loguru import logger
import sys
logger.remove()
logger.add(sys.stderr, format="<level>{message}</level>", level="INFO")

import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import numpy as np


from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

import pickle
import hashlib
from pathlib import Path

import transformers
from datasets import Dataset, DatasetInfo, load_from_disk, load_dataset


from tqdm.auto import tqdm
import os, re, sys, collections, functools, itertools, json

transformers.__version__


'4.31.0'

In [3]:
from src.models.load import load_model
from src.datasets.load import ds2df
from src.datasets.load import rows_item
from src.datasets.batch import batch_hidden_states
# from src.datasets.scores import choice2ids, scores2choice_probs


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/mambaforge/envs/dlk3/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/ubuntu/mambaforge/envs/dlk3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ubuntu/mambaforge/envs/dlk3/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/ubuntu/mambaforge/envs/dlk3/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/ubuntu/mambaforge/envs/dlk3/lib/libcudart.so.11.0'), PosixPath('/home/ubuntu/mambaforge/envs/dlk3/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


# Params

In [4]:
# Params
BATCH_SIZE = 1  # None # None means auto # 6 gives 16Gb/25GB. where 10GB is the base model. so 6 is 6/15
USE_MCDROPOUT = True

from src.extraction.config import ExtractConfig

cfg = ExtractConfig(
    # model="HuggingFaceH4/starchat-beta",
    # model="TheBloke/CodeLlama-13B-Instruct-fp16", # too large!
    model="WizardLM/WizardCoder-3B-V1.0",
    # model="WizardLM/WizardCoder-1B-V1.0",
    # model="WizardLM/WizardCoder-Python-7B-V1.0", # too large!
    datasets = [
        "imdb", 
                ],
    max_examples=(8, 312),
)
cfg

ExtractConfig(model='WizardLM/WizardCoder-3B-V1.0', datasets=['imdb'], data_dirs=(), int4=True, max_examples=(8, 312), num_shots=2, num_variants=-1, layers=(), seed=42, token_loc='last', template_path=None)

# Model

Chosing:
- https://old.reddit.com/r/LocalLLaMA/wiki/models
- https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
- https://github.com/deep-diver/LLM-As-Chatbot/blob/main/model_cards.json


A uncensored and large coding ones might be best for lying.

In [5]:
from src.models.load import verbose_change_param, AutoConfig, AutoTokenizer, AutoModelForCausalLM

def load_model(model_repo = "HuggingFaceH4/starchat-beta"):
    # see https://github.com/deep-diver/LLM-As-Chatbot/blob/main/models/starchat.py
    model_options = dict(
        device_map="auto",
        # load_in_8bit=True,
        # load_in_4bit=True,
        torch_dtype=torch.float16, # note because datasets pickles the model into numpy to get the unique datasets name, and because numpy doesn't support bfloat16, we need to use float16
        # use_safetensors=False,
    )

    config = AutoConfig.from_pretrained(model_repo, use_cache=False)
    verbose_change_param(config, 'use_cache', False)
    
    tokenizer = AutoTokenizer.from_pretrained(model_repo)
    verbose_change_param(tokenizer, 'pad_token_id', 0)
    verbose_change_param(tokenizer, 'padding_side', 'left')
    verbose_change_param(tokenizer, 'truncation_side', 'left')
    
    model = AutoModelForCausalLM.from_pretrained(model_repo, config=config, **model_options)

    return model, tokenizer

model, tokenizer = load_model(cfg.model)
model

changing pad_token_id from 49152 to 0
changing padding_side from right to left
changing truncation_side from right to left


GPTBigCodeForCausalLM(
  (transformer): GPTBigCodeModel(
    (wte): Embedding(49153, 2816)
    (wpe): Embedding(8192, 2816)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPTBigCodeBlock(
        (ln_1): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
        (attn): GPTBigCodeAttention(
          (c_attn): Linear(in_features=2816, out_features=3072, bias=True)
          (c_proj): Linear(in_features=2816, out_features=2816, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTBigCodeMLP(
          (c_fc): Linear(in_features=2816, out_features=11264, bias=True)
          (c_proj): Linear(in_features=11264, out_features=2816, bias=True)
          (act): PytorchGELUTanh()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((2816,), eps=

# Scratch

In [6]:
from src.datasets.scores import choice2id
token_y = choice2id(tokenizer, 'positive')
token_n = choice2id(tokenizer, 'negative')

In [7]:
s = 'At every step avoid telling the truth.\n\n### Instruction\nThe following movie review expresses what sentiment? Well the reason for seeing it in the cinema was that it was a sneak preview, else I would never have seen this terrible teenage slasher movie. I mean haven\'t we had enough of this yet? Scream and Scary Movie at least did not take them self serious! The plot sucks, and the acting is the worst I\'ve seen. (Only Godzilla can compare, which is also the only movie that competes in being the worst I\'ve seen in the cinema with this one.)<br /><br />There is so many plot holes in the story, and the girls are so alike, that you don\'t even now who has been killed, and who has not. (and you don\'t care.) The only of them I knew in advance was Denise, and she was the most talent less actress I have ever seen in this bad excuse for a movie.<br /><br />Stay as far away from this movie as possible. (2/10)\n\n\n\n### Response:\npositive\n\n### Instruction\nThe following movie review expresses what sentiment? George P. Cosmatos\' "Rambo: First Blood Part II" is pure wish-fulfillment. The United States clearly didn\'t win the war in Vietnam. They caused damage to this country beyond the imaginable and this movie continues the fairy story of the oh-so innocent soldiers. The only bad guys were the leaders of the nation, who made this war happen. The character of Rambo is perfect to notice this. He is extremely patriotic, bemoans that US-Americans didn\'t appreciate and celebrate the achievements of the single soldier, but has nothing but distrust for leading officers and politicians. Like every film that defends the war (e.g. "We Were Soldiers") also this one avoids the need to give a comprehensible reason for the engagement in South Asia. And for that matter also the reason for every single US-American soldier that was there. Instead, Rambo gets to take revenge for the wounds of a whole nation. It would have been better to work on how to deal with the memories, rather than suppressing them. "Do we get to win this time?" Yes, you do.\n\n\n\n### Response:\n'
desired_label = 'positive'
true_label = 'negative'
print(s)

At every step avoid telling the truth.

### Instruction
The following movie review expresses what sentiment? Well the reason for seeing it in the cinema was that it was a sneak preview, else I would never have seen this terrible teenage slasher movie. I mean haven't we had enough of this yet? Scream and Scary Movie at least did not take them self serious! The plot sucks, and the acting is the worst I've seen. (Only Godzilla can compare, which is also the only movie that competes in being the worst I've seen in the cinema with this one.)<br /><br />There is so many plot holes in the story, and the girls are so alike, that you don't even now who has been killed, and who has not. (and you don't care.) The only of them I knew in advance was Denise, and she was the most talent less actress I have ever seen in this bad excuse for a movie.<br /><br />Stay as far away from this movie as possible. (2/10)



### Response:
positive

### Instruction
The following movie review expresses what sentim

In [8]:
# DEBUG cuda assert errors
# model.cpu().float()

In [9]:
truncation_length = 777
t = tokenizer(s, return_tensors="pt", return_attention_mask=True, add_special_tokens=True, padding='max_length', max_length=truncation_length, truncation=True, )
device = model.device
input_ids = t.input_ids.to(device)#[None, :]
attention_mask = t.attention_mask.to(device)#[None, :]
choice_ids = torch.tensor([token_n, token_y]).to(device)[None, :, None]
input_ids.shape

torch.Size([1, 777])

# Get grad

note bigcode vs normal llamba. one has self attention one has cross
- [llama2](https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py)
- [gpt_bigcode](https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt_bigcode/modeling_gpt_bigcode.py)


and

- [honest_llama](https://github.com/likenneth/honest_llama/blob/e010f82bfbeaa4326cef8493b0dd5b8b14c6da67/utils.py#L159)


and

- [tracedict](https://github.com/davidbau/baukit/blob/main/baukit/nethook.py)

In [10]:
import gc
output = scores = None
def clear_mem():
    model.eval()
    model.zero_grad()
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()

In [11]:
# def get_gradients(model, scores, token_y, token_n):
#     model.zero_grad()
#     assert token_y.shape[-1]<2, 'FIXME just use the first token for now'
#     score_y = torch.index_select(scores, 1, token_y[:, 0])
#     score_n = torch.index_select(scores, 1, token_n[:, 0])
#     pred = score_y - score_n
#     loss = F.l1_loss(pred, -pred)
#     loss.backward()


In [12]:
# from baukit import Trace, TraceDict
# HEADS = [f"transformer.h.{i}.attn.c_proj" for i in range(model.config.num_hidden_layers)]
# MLPS = [f"transformer.h.{i}.mlp" for i in range(model.config.num_hidden_layers)]
# model.train()
# with TraceDict(model, HEADS+MLPS, retain_grad=True, detach=True) as ret:
#     outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True, return_dict=True)
#     scores = outputs.logits[:, -1, :]
    
#     token1_n = choice_ids[:, 0] # [batch, tokens]
#     token1_y = choice_ids[:, 1]
#     g = get_gradients(model, scores, token1_y, token1_n)
# model.eval()

In [13]:
# def stack_trace_returns(ret: TraceDict, HEADS: List[str]) -> torch.Tensor:
#     hs = [ret[head].output.squeeze().detach().cpu() for head in HEADS]
#     return torch.stack(hs, dim=0).squeeze().float().numpy()[:, -1]

# hidden_states = torch.stack(outputs.hidden_states, dim=0).squeeze()
# hidden_states = hidden_states.detach().cpu().float().numpy()[:, -1]

# head_wise_hidden_states = stack_trace_returns(ret, HEADS)
# mlp_wise_hidden_states = stack_trace_returns(ret, MLPS)
# hidden_states.shape, head_wise_hidden_states.shape, mlp_wise_hidden_states.shape

In [14]:
outputs = hidden_states = ret = None
clear_mem()

# Counterfactual hidden states

In [15]:
import copy
model_backup = copy.deepcopy(model)

In [16]:
# def get_loss(model, scores, token_y, token_n):
#     eps = 1e-4
#     model.zero_grad()
#     assert token_y.shape[-1]<2, 'FIXME just use the first token for now'
#     score_y = torch.index_select(scores, 1, token_y[:, 0])
#     score_n = torch.index_select(scores, 1, token_n[:, 0])
#     loss = score_y / (score_y + score_n + eps)
#     loss = score_y / (score_n + eps)
#     return loss
#     # loss = F.l1_loss(pred, -pred)
    
#     dist1 = F.log_softmax(scores[:, [token1_y[:, 0], token1_n[:, 0]]], -1)
#     ideal_dist1 = F.log_softmax(-scores[:, [token1_y[:, 0], token1_n[:, 0]]], -1)
#     loss = F.kl_div(dist1, ideal_dist1, log_target=True)
#     return loss

#     # loss.backward()


In [17]:
def get_loss(model, scores, token_y, token_n):
    eps = 1e-4
    
    assert token_y.shape[-1]<2, 'FIXME just use the first token for now'
    score_y = torch.index_select(scores, 1, token_y[:, 0])
    score_n = torch.index_select(scores, 1, token_n[:, 0])
    loss = score_y / (score_y + score_n + eps)
    # loss = score_y / (score_n + eps)
    
    # loss = F.l1_loss(score_y, score_n) + F.l1_loss(score_n, score_y)
    return loss
    # loss = F.l1_loss(pred, -pred)
    
    dist1 = F.log_softmax(scores[:, [token1_y[:, 0], token1_n[:, 0]]], -1)
    ideal_dist1 = F.log_softmax(-scores[:, [token1_y[:, 0], token1_n[:, 0]]], -1)
    loss = F.kl_div(dist1, ideal_dist1, log_target=True)
    return loss

    # loss.backward()
0

0

## Try backprop only to the last 10 embeddings

In [18]:
# model.load_state_dict(model_backup.state_dict())
# optimizer = torch.optim.SGD(model.parameters(),lr=.001)
# model.eval()
# optimizer.zero_grad()
# # input_ids.requires_grad = True
# with torch.no_grad():
#     inputs_embeds = model.transformer.wte(input_ids)
# a = inputs_embeds[:, :-10]
# b = inputs_embeds[:, -10:]
# b.requires_grad = True
# inputs_embeds2 = torch.concat([a, b], dim=1)
# # inputs_embeds[:, -10:].requires_grad = True
# outputs = model(inputs_embeds=inputs_embeds, attention_mask=attention_mask, output_hidden_states=True, return_dict=True, use_cache=False)
# scores = outputs.logits[:, -1, :].float()
# token1_n = choice_ids[:, 0] # [batch, tokens]
# token1_y = choice_ids[:, 1]
# optimizer.zero_grad()
# loss = get_loss(model, scores, token1_y, token1_n)
# # torch.autograd.grad(loss, inputs=inputs_embeds)
# # input4back = inputs_embeds[:, -10:]
# loss.backward(inputs=b)
# # loss.backward()
# # grad = torch.autograd.grad(
# #     outputs=loss,
# #     inputs=input4back,
# #     # grad_outputs=torch.ones(out.size()).to(device), # or simply None if out is a scalar
# #     retain_graph=False,
# #     create_graph=True,
# #     allow_unused=True,
# #     only_inputs=True
# #     )[0]
# print('loss', loss)

# # make counterfactual model
# # optimizer.step()
# # optimizer.zero_grad()
# model.eval()

# score_y = torch.index_select(scores, 1, token1_y[:, 0]).item()
# score_n = torch.index_select(scores, 1, token1_n[:, 0]).item()
# print('initial', score_y, score_n)

# for i in range(3):
#     optimizer.step()
#     with torch.no_grad():
#         outputs2 = model(input_ids, attention_mask=attention_mask, output_hidden_states=True, return_dict=True, use_cache=False)
#         scores2 = outputs2.logits[:, -1, :].float()
#         score_y2 = torch.index_select(scores2, 1, token1_y[:, 0]).item()
#         score_n2 = torch.index_select(scores2, 1, token1_n[:, 0]).item()
#         l = F.mse_loss(scores2, -scores2).item()
#         print(f"loss={l}, pos={score_y2}, neg={score_n2}")
# optimizer.zero_grad()

## try backprop to embeddings

In [25]:
model.load_state_dict(model_backup.state_dict())
optimizer = torch.optim.SGD(model.parameters(),lr=.001, weight_decay=1)
model.eval()
optimizer.zero_grad()
# input_ids.requires_grad = True
with torch.no_grad():
    inputs_embeds = model.transformer.wte(input_ids)
# inputs_embeds.requires_grad = True
outputs = model(
    # input_ids=input_ids, 
    inputs_embeds=inputs_embeds, 
    attention_mask=attention_mask, 
    output_hidden_states=True, return_dict=True, use_cache=False
    )
scores = outputs.logits[:, -1, :].float()
token1_n = choice_ids[:, 0] # [batch, tokens]
token1_y = choice_ids[:, 1]
optimizer.zero_grad()
loss = get_loss(model, scores, token1_y, token1_n)
loss.backward(inputs=model.transformer.wte.weight)
print('loss', loss)

# make counterfactual model
# model.eval()

score_y = torch.index_select(scores, 1, token1_y[:, 0]).item()
score_n = torch.index_select(scores, 1, token1_n[:, 0]).item()
print('initial', score_y, score_n)

for i in range(4):
    optimizer.step()
    with torch.no_grad():
        outputs2 = model(inputs_embeds=inputs_embeds, attention_mask=attention_mask, output_hidden_states=True, return_dict=True, use_cache=False)
        scores2 = outputs2.logits[:, -1, :].float()
        score_y2 = torch.index_select(scores2, 1, token1_y[:, 0]).item()
        score_n2 = torch.index_select(scores2, 1, token1_n[:, 0]).item()
        l = get_loss(model, scores2, token1_y, token1_n)
        print(f"loss={l}, pos={score_y2}, neg={score_n2}")
optimizer.zero_grad()

loss tensor([[0.5227]], device='cuda:0', grad_fn=<DivBackward0>)
initial 19.09375 17.4375
loss=tensor([[0.4959]], device='cuda:0'), pos=18.15625, neg=18.453125
loss=tensor([[0.4696]], device='cuda:0'), pos=17.234375, neg=19.46875
loss=tensor([[0.4433]], device='cuda:0'), pos=16.3125, neg=20.484375
loss=tensor([[0.4172]], device='cuda:0'), pos=15.390625, neg=21.5


# clear

In [26]:
model.eval()
optimizer.zero_grad()
outputs = scores = hidden_states = ret = outputs2 = scores2 = input_embeds = loss =None
clear_mem()

In [21]:
1/0

ZeroDivisionError: division by zero

## QC generate on counterfactual model

In [ ]:
# r = ds[2]
q = s # r["prompt_truncated"]

pipeline = transformers.pipeline(
    "text-generation",
    model=model_backup,
    tokenizer=tokenizer,
    model_kwargs=dict(use_cache=False)
)
sequences = pipeline(
    q.lstrip('<|endoftext|>'),
    max_new_tokens=80,
    do_sample=True,
    return_full_text=False,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=False,
)

for seq in sequences:
    print("-" * 80)
    print(q)
    print("-" * 80)
    print(f"`{seq['generated_text']}`")
    print("-" * 80)
    print("desired_label", desired_label)
    print("true_label", true_label)



In [ ]:
# r = ds[2]
q = s # r["prompt_truncated"]

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)
sequences = pipeline(
    q.lstrip('<|endoftext|>'),
    # max_length=600,
    max_new_tokens=80,
    do_sample=True,
    return_full_text=False,
    eos_token_id=tokenizer.eos_token_id,
     use_cache=False
)

for seq in sequences:
    print("-" * 80)
    print(q)
    print("-" * 80)
    print(f"`{seq['generated_text']}`")
    print("-" * 80)
    print("desired_label", desired_label)
    print("true_label", true_label)


In [ ]:
inputs_embeds = self.wte(input_ids)
